<a href="https://colab.research.google.com/github/cinnData/DataSci/blob/main/Notebooks/retail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [DS-06E] Example - Online retail transaction data

## Importing the data

In [ ]:
import pandas as pd
path = 'https://raw.githubusercontent.com/cinnData/DataSci/main/Data/'
filename = path + 'retail.csv.zip'
df = pd.read_csv(filename, parse_dates=[4])

## Exploring the data

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
pd.crosstab(df['InvoiceNo'].str.contains('C'), df['Quantity'] < 0)

## Q1. New column with the number of days since the invoice was generated

In [ ]:
max_date = df['InvoiceDate'].max()
max_date

In [ ]:
df['Diff'] = max_date - df['InvoiceDate']
df['Diff']

In [ ]:
df['Diff'] = df['Diff'].dt.days
df.head()

## Q2. Group by customer and aggregate to create the RFM data set

In [ ]:
RF = df.groupby('CustomerID')['Diff'].agg(['min', 'count'])
RF.head()

In [ ]:
RF.columns = ['Recency', 'Frequency']

In [ ]:
df['Monetary'] = df['Quantity']*df['UnitPrice']

In [ ]:
M = df.groupby('CustomerID')['Monetary'].sum()
M.head()

In [ ]:
RFM = RF.merge(M, left_index=True, right_index=True)
RFM.head()

# Q3. 8-cluster analysis

In [ ]:
RFM.describe()

In [ ]:
def normalize(x): return (x - x.min())/(x.max() - x.min())

In [ ]:
RFM1 = RFM.apply(normalize, axis=0)
RFM1.head()

In [ ]:
import scipy.cluster.vq as cluster

In [ ]:
centers = cluster.kmeans(RFM1, k_or_guess=8)[0]
centers

In [ ]:
labels = cluster.vq(RFM1, centers)[0]
labels

In [ ]:
RFM['Segment'] = labels
RFM.head()

In [ ]:
RFM['Segment'].value_counts()